# Análise de Sentimentos

Treinando a machine learning, utilizando o dataset recebido por email. (Faça o download e coloque a pasta dentro do repositorio)

In [1]:
import os
import xmltodict
import pandas as pd
os.chdir('..')
from nlputils.lexical.normalizer import Normalizer

Pegando todos os arquivos xml do trainset

In [2]:
dataset_path = 'trainset/'
all_files = []
for directory in os.listdir(dataset_path):
    if directory != '.DS_Store':
        for stars in os.listdir(dataset_path + directory):
            for file in os.listdir(dataset_path + directory + '/' + stars):
                if 'xml' in file:
                    all_files.append('{}{}/{}/{}'.format(dataset_path, directory, stars, file))

Criando a estrutura que será utilizada pela Machine Learning

In [72]:
dataset = {'trainset':[], 'polarity':[], 'bin_polarity': [], 'review':[]}
for file in all_files:
    filename = file.split('/')[-1]
    with open(file, 'r') as text_file:
        data = text_file.read()
        try:
            dict_data = xmltodict.parse(data)
            dataset['trainset'].append(filename)
            polarity = float(dict_data['review']['stars']['@value'])
            bin_polarity = 1 if polarity > 5 else 0  # transform into binary polarity
            dataset['bin_polarity'].append(bin_polarity)
            dataset['polarity'].append(polarity)
            dataset['review'].append(str(dict_data['review']['opinion']))
        except:
            print('Arquivo {} está com xml mal formatado'.format(file))

In [73]:
dataframe = pd.DataFrame(data=dataset)
dataframe.head()

,bin_polarity,polarity,review,trainset
0,0,3.0,Se não fosse o uniforme do palmeiras seria uma...,0_189781.xml
1,0,3.0,BBB\n\nO que gostei: GGG\n\nO que não gostei: ...,0_366331.xml
2,0,3.0,Um bom produto por um preço de melhor ainda.\n...,0_308224.xml
3,0,3.0,Essa camiseta do time esfecifico é muito boa p...,2_170381.xml
4,0,3.0,"NÃO TENHO MUiTO O QUE DiZER,AFiNAL COMPREi PAR...",0_458303.xml


Agora vamos ler todos os textos e adicionar um novo campo na dataframe que irá ter a sentença sem pontuações e stopwords

In [77]:
from nltk.corpus import stopwords
normalizer = Normalizer()

def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

dataframe['normalized_review'] = dataframe['review'].apply(preprocessing)
dataframe.head()

,bin_polarity,polarity,review,trainset,normalized_review
0,0,3.0,Se não fosse o uniforme do palmeiras seria uma...,0_189781.xml,uniforme palmeiras camisa bonita comprar cores...
1,0,3.0,BBB\n\nO que gostei: GGG\n\nO que não gostei: ...,0_366331.xml,bbb gostei ggg gostei gggg
2,0,3.0,Um bom produto por um preço de melhor ainda.\n...,0_308224.xml,bom produto preço melhor ainda gostei bom prod...
3,0,3.0,Essa camiseta do time esfecifico é muito boa p...,2_170381.xml,camiseta time esfecifico é boa causa material ...
4,0,3.0,"NÃO TENHO MUiTO O QUE DiZER,AFiNAL COMPREi PAR...",0_458303.xml,dizerafinal comprei presente


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

# pegando so indice par
train_reviews = dataframe['normalized_review'].values.tolist()[::2]
train_classes = dataframe['bin_polarity'].values.tolist()[::2]

# pegando so indice impar
test_reviews = dataframe['normalized_review'].values.tolist()[1::2] 
test_classes = dataframe['bin_polarity'].values.tolist()[1::2]

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

Treinando 

In [92]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classifier = SVC()
classifier.fit(X, train_classes)

ValueError: The number of classes has to be greater than one; got 1 class